<a href="https://colab.research.google.com/github/stevegabriel1/adventures_in_time_series/blob/master/afl_data_processing_mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Required initial steps:
    
    1. The text file from https://afltables.com/afl/stats/biglists/bg3.txt is saved as llist.txt within the data_projects/sports_data folder.
    
    2. The file is then processed through the bash script file a_updater.sh to convert it to a csv: Type the following in Terminal:  ./a_updater.sh [from the data_projects/sports_data directory]
    
    3. Copy to afl_dataexploration subfolder. The bash file saves the processed file as newlist.csv, ready to be imported from within Jupyter as below.

In [ ]:
import urllib.request    
urllib.request.urlretrieve("https://afltables.com/afl/stats/biglists/bg3.txt", "llist.txt")

('llist.txt', <http.client.HTTPMessage at 0x105f06af0>)

### **Now, run a_updater.sh from the command line as above**

In [ ]:
import os

In [ ]:
os.getcwd()

'/Users/steve_gabriel/Documents/data_projects/sports_etc/afl_allgames/afl_explore'

In [ ]:
import pandas as pd
cols = ['seq','date','round','team1','gls_1','bhs_1','pts_1','team2','gls_2','bhs_2','pts_2','venue']

In [ ]:
upd_round = pd.read_csv("newlist.csv", names = cols)
trans = upd_round

In [ ]:
# Modify index to start at 1 
trans.index += 1
trans.tail(9)

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue
15966,15966,20-Aug-2021,R23,Western Bulldogs,10,4,64,Port Adelaide,9,12,66,Docklands
15967,15967,21-Aug-2021,R23,Richmond,12,11,83,Hawthorn,12,11,83,M.C.G.
15968,15968,21-Aug-2021,R23,Sydney,21,10,136,Gold Coast,6,13,49,Docklands
15969,15969,21-Aug-2021,R23,Brisbane Lions,19,11,125,West Coast,13,9,87,Gabba
15970,15970,21-Aug-2021,R23,Geelong,12,5,77,Melbourne,12,9,81,Kardinia Park
15971,15971,21-Aug-2021,R23,Carlton,11,9,75,GW Sydney,12,17,89,Docklands
15972,15972,22-Aug-2021,R23,St Kilda,17,5,107,Fremantle,6,13,49,Bellerive Oval
15973,15973,22-Aug-2021,R23,Essendon,16,6,102,Collingwood,9,10,64,M.C.G.
15974,15974,22-Aug-2021,R23,Adelaide,13,20,98,North Melbourne,8,6,54,Adelaide Oval


In [ ]:
# Add states
nsw = ['S.C.G.','Sydney Showground','Stadium Australia','Blacktown','Albury']
qld = ['Gabba','Carrara','Cazaly\'s Stadium','Brisbane Exhibition','Riverway Stadium']
sa = ['Football Park','Adelaide Oval']
wa = ['W.A.C.A.','Perth Stadium','Subiaco']
nt = ['Marrara Oval','Traeger Park']
tas = ['York Park','Bellerive Oval','North Hobart']
act = ['Manuka Oval','Bruce Stadium']
intl = ['Wellington','Jiangwan Stadium']

In [ ]:
def locations(x):
    if x in nsw:
        return "NSW"
    elif x in sa:
        return "SA"
    elif x in wa:
        return "WA"
    elif x in nt:
        return "NT"
    elif x in tas:
        return "TAS"
    elif x in intl:
        return "INTL"
    elif x in act:
        return "ACT"
    elif x in qld:
        return "QLD"
    else: 
        return "VIC"

In [ ]:
trans["state"] = trans["venue"].apply(locations)
trans["state"].value_counts()

VIC     13266
WA        704
SA        637
QLD       605
NSW       563
TAS       111
ACT        53
NT         29
INTL        6
Name: state, dtype: int64

### Check that the most recent round is included:

In [ ]:
trans.tail(9)

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue,state
15966,15966,20-Aug-2021,R23,Western Bulldogs,10,4,64,Port Adelaide,9,12,66,Docklands,VIC
15967,15967,21-Aug-2021,R23,Richmond,12,11,83,Hawthorn,12,11,83,M.C.G.,VIC
15968,15968,21-Aug-2021,R23,Sydney,21,10,136,Gold Coast,6,13,49,Docklands,VIC
15969,15969,21-Aug-2021,R23,Brisbane Lions,19,11,125,West Coast,13,9,87,Gabba,QLD
15970,15970,21-Aug-2021,R23,Geelong,12,5,77,Melbourne,12,9,81,Kardinia Park,VIC
15971,15971,21-Aug-2021,R23,Carlton,11,9,75,GW Sydney,12,17,89,Docklands,VIC
15972,15972,22-Aug-2021,R23,St Kilda,17,5,107,Fremantle,6,13,49,Bellerive Oval,TAS
15973,15973,22-Aug-2021,R23,Essendon,16,6,102,Collingwood,9,10,64,M.C.G.,VIC
15974,15974,22-Aug-2021,R23,Adelaide,13,20,98,North Melbourne,8,6,54,Adelaide Oval,SA


## 1. HVAR & AGGR

In [ ]:
trans['hvar'] = trans['pts_1'] - trans['pts_2']
trans['aggr'] = trans['pts_1'] + trans['pts_2'] 

## 2. YEAR & MONTH

In [ ]:
import time
import datetime

In [ ]:
trans['date'] = pd.to_datetime(trans['date'])
trans['year'], trans['month'] = trans['date'].dt.year, trans['date'].dt.month

## 3. ABBREVIATED CLUB NAMES

In [ ]:
adelaide = ['Adelaide']
brisbane = ['Brisbane Bears','Brisbane Lions']
carlton = ['Carlton']
collingwood = ['Collingwood']
essendon = ['Essendon']
fitzroy = ['Fitzroy']
footscray = ['Footscray']
fremantle = ['Fremantle']
geelong = ['Geelong']
gold_coast = ['Gold Coast']
gws = ['GW Sydney']
hawthorn = ['Hawthorn']
melbourne = ['Melbourne']
north_melbourne = ['North Melbourne','Kangaroos']
port_adelaide = ['Port Adelaide']
richmond = ['Richmond']
south_melbourne = ['South Melbourne']
st_kilda = ['St Kilda']
sydney = ['Sydney']
west_coast = ['West Coast']
western_bulldogs = ['Western Bulldogs', 'Footscray']
university = ['University']

In [ ]:
def team_shorten(x):
    if x in adelaide:
        return "AD"
    elif x in brisbane:
        return "BR"
    elif x in carlton:
        return "CA"
    elif x in collingwood:
        return "CO"
    elif x in essendon:
        return "ES"
    elif x in fitzroy:
        return "FI"
    elif x in fremantle:
        return "FR"
    elif x in geelong:
        return "GE"
    elif x in gold_coast:
        return "GC"
    elif x in gws:
        return "GW"
    elif x in hawthorn:
        return "HA"
    elif x in melbourne:
        return "ME"
    elif x in north_melbourne:
        return "NM"
    elif x in port_adelaide:
        return "PA"
    elif x in richmond:
        return "RI"
    elif x in south_melbourne:
        return "SM"
    elif x in st_kilda:
        return "SK"
    elif x in sydney:
        return "SY"
    elif x in west_coast:
        return "WC"
    elif x in western_bulldogs:
        return "WB"
    elif x in university:
        return "UN"
    else: 
        return "NOT_FOUND"

In [ ]:
trans["t1"] = trans["team1"].apply(team_shorten)
trans["t2"] = trans["team2"].apply(team_shorten)

In [ ]:
trans['t1'].value_counts().sum()

15974

# 4. FIX TEAM NAMES ANOMOLIES

* Merge Footscray & Western Bulldogs as **Western Bulldogs**
* Merge North Melbourne with Kangaroos as **North Melbourne**
* Merge Brisbane Lions with Brisbane Bears as **Brisbane**
* Leave Sydney and **South Melbourne** as separate
* Leave **Fitzroy** as separate
* Modify GW Sydney to **GWS Giants**

In [ ]:
# Function from Analytics Vidhya
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

In [ ]:
trans["team1"] = coding(trans["team1"], {'Footscray':'Western Bulldogs'})
trans["team2"] = coding(trans["team2"], {'Footscray':'Western Bulldogs'})
trans["team1"] = coding(trans["team1"], {'Kangaroos':'North Melbourne'})
trans["team2"] = coding(trans["team2"], {'Kangaroos':'North Melbourne'})
trans["team1"] = coding(trans["team1"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team2"] = coding(trans["team2"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team1"] = coding(trans["team1"], {'GW Sydney':'GWS Giants'})
trans["team2"] = coding(trans["team2"], {'GW Sydney':'GWS Giants'})

In [ ]:
trans["team2"].value_counts()

Collingwood         1300
Carlton             1271
Essendon            1254
Geelong             1236
Melbourne           1234
St Kilda            1225
Richmond            1164
North Melbourne     1017
Hawthorn            1016
Western Bulldogs    1008
Fitzroy              971
South Melbourne      783
Sydney               460
West Coast           405
Brisbane             392
Adelaide             358
Fremantle            301
Port Adelaide        283
Gold Coast           117
GWS Giants           116
University            63
Name: team2, dtype: int64

In [ ]:
trans["venue"].value_counts().tail(15)

Coburg Oval            9
Eureka Stadium         8
Yarraville Oval        7
Traeger Park           7
North Hobart           5
Wellington             3
Jiangwan Stadium       3
Olympic Park           3
Bruce Stadium          1
Brisbane Exhibition    1
Albury                 1
Yallourn               1
Euroa                  1
Blacktown              1
Riverway Stadium       1
Name: venue, dtype: int64

# 5. ALL DRAWS (BOOLEAN)

In [ ]:
def reg_draw(hvar):
    if hvar == 0:
        return 1
    else: return 0

In [ ]:
trans["draw_rt"] = trans["hvar"].apply(reg_draw)

In [ ]:
trans['draw_rt'].value_counts()

0    15811
1      163
Name: draw_rt, dtype: int64

### The ET draws

In [ ]:
def et_draw(seq):
    if seq == 10794:
        return 1
    elif seq == 13203:
        return 1
    else: return 0

In [ ]:
trans["etime_draw"] = trans["seq"].apply(et_draw)
trans['etime_draw'].value_counts()

0    15972
1        2
Name: etime_draw, dtype: int64

In [ ]:
### Add the two columns
trans['draws_all']  = trans['draw_rt'] + trans['etime_draw']
trans['draws_all'].value_counts()

0    15809
1      165
Name: draws_all, dtype: int64

In [ ]:
# Check
trans.iloc[:,:][(trans.etime_draw == 1)]

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,state,hvar,aggr,year,month,t1,t2,draw_rt,etime_draw,draws_all
10794,10794,1994-09-10,QF,North Melbourne,15,24,114,Hawthorn,13,13,...,VIC,23,205,1994,9,NM,HA,0,1,1
13203,13203,2007-09-14,SF,West Coast,10,14,74,Collingwood,13,15,...,WA,-19,167,2007,9,WC,CO,0,1,1


# 6. GAME CATEGORY

In [ ]:
finals = ['EF','QF','SF','PF','GF']

In [ ]:
def game_cat(x):
    if x in finals:
        return "final"
    else: 
        return "reg_season"

In [ ]:
trans["type"] = trans["round"].apply(game_cat)

In [ ]:
trans['type'].value_counts()

reg_season    15301
final           673
Name: type, dtype: int64

In [ ]:
trans.tail()

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,hvar,aggr,year,month,t1,t2,draw_rt,etime_draw,draws_all,type
15970,15970,2021-08-21,R23,Geelong,12,5,77,Melbourne,12,9,...,-4,158,2021,8,GE,ME,0,0,0,reg_season
15971,15971,2021-08-21,R23,Carlton,11,9,75,GWS Giants,12,17,...,-14,164,2021,8,CA,GW,0,0,0,reg_season
15972,15972,2021-08-22,R23,St Kilda,17,5,107,Fremantle,6,13,...,58,156,2021,8,SK,FR,0,0,0,reg_season
15973,15973,2021-08-22,R23,Essendon,16,6,102,Collingwood,9,10,...,38,166,2021,8,ES,CO,0,0,0,reg_season
15974,15974,2021-08-22,R23,Adelaide,13,20,98,North Melbourne,8,6,...,44,152,2021,8,AD,NM,0,0,0,reg_season


## 7. GROUPING COLUMNS

In [ ]:
def grouping_2K(seq):
    n=500
    if seq <= 4*n:
        return 1
    elif seq <= 8*n:
        return 2
    elif seq <= 12*n:
        return 3
    elif seq <= 16*n:
        return 4
    elif seq <= 20*n:
        return 5
    elif seq <= 24*n:
        return 6
    elif seq <= 28*n:
        return 7
    else: return 8

In [ ]:
trans["grp_2K"] = trans["seq"].apply(grouping_2K)

In [ ]:
def grouping_1K(seq):
    n=1000
    if seq <= n:
        return 1
    elif seq <= 2*n:
        return 2
    elif seq <= 3*n:
        return 3
    elif seq <= 4*n:
        return 4
    elif seq <= 5*n:
        return 5
    elif seq <= 6*n:
        return 6
    elif seq <= 7*n:
        return 7
    elif seq <= 8*n:
        return 8
    elif seq <= 9*n:
        return 9
    elif seq <= 10*n:
        return 10
    elif seq <= 11*n:
        return 11
    elif seq <= 12*n:
        return 12
    elif seq <= 13*n:
        return 13
    elif seq <= 14*n:
        return 14
    elif seq <= 15*n:
        return 15
    else: return 16

In [ ]:
trans["grp_1K"] = trans["seq"].apply(grouping_1K)

## 8. ADD res1 and res2 columns as categorical 1 = win, 2 = loss, 3 = draw based on hvar (ignoring the two extra time finals)

In [ ]:
def result_cat_h(hvar):
    if hvar > 0:
        return 1
    elif hvar < 0:
        return 2
    else: return 3

In [ ]:
def result_cat_a(hvar):
    if hvar > 0:
        return 2
    elif hvar < 0:
        return 1
    else: return 3

In [ ]:
trans['res1'] = trans['hvar'].apply(result_cat_h)
trans['res2'] = trans['hvar'].apply(result_cat_a)

In [ ]:
trans.head()

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,t1,t2,draw_rt,etime_draw,draws_all,type,grp_2K,grp_1K,res1,res2
1,1,1897-05-08,R1,Fitzroy,6,13,49,Carlton,2,4,...,FI,CA,0,0,0,reg_season,1,1,1,2
2,2,1897-05-08,R1,Collingwood,5,11,41,St Kilda,2,4,...,CO,SK,0,0,0,reg_season,1,1,1,2
3,3,1897-05-08,R1,Geelong,3,6,24,Essendon,7,5,...,GE,ES,0,0,0,reg_season,1,1,2,1
4,4,1897-05-08,R1,South Melbourne,3,9,27,Melbourne,6,8,...,SM,ME,0,0,0,reg_season,1,1,2,1
5,5,1897-05-15,R2,South Melbourne,6,4,40,Carlton,5,6,...,SM,CA,0,0,0,reg_season,1,1,1,2


## Export wide format results since 1897

In [ ]:
trans.to_csv('wide_afl_run.csv')

# PAGEBREAK

# Convert data frame from wide to long format - each game appears twice (once for each team)

## 9. Reduce and re-order the data frame

In [ ]:
tracker = trans
tracker = tracker[['year', 'team1','pts_1','team2','pts_2','venue','t1','t2','hvar']]
tracker.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar
15970,2021,Geelong,77,Melbourne,81,Kardinia Park,GE,ME,-4
15971,2021,Carlton,75,GWS Giants,89,Docklands,CA,GW,-14
15972,2021,St Kilda,107,Fremantle,49,Bellerive Oval,SK,FR,58
15973,2021,Essendon,102,Collingwood,64,M.C.G.,ES,CO,38
15974,2021,Adelaide,98,North Melbourne,54,Adelaide Oval,AD,NM,44


### Create two new columns with values 1 and 0 for the two subsets

In [ ]:
tracker['subset_1'] = 1
tracker['subset_2'] = 0

/var/folders/p7/vz1lk6y57ynbc3fpjldpnb700000gn/T/ipykernel_45332/4012781384.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracker['subset_1'] = 1
/var/folders/p7/vz1lk6y57ynbc3fpjldpnb700000gn/T/ipykernel_45332/4012781384.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracker['subset_2'] = 0


In [ ]:
tracker.head()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_1,subset_2
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,0
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,0
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,0
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1,0
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1,0


### List for dropping 

In [ ]:
droplist_1 = ['subset_2']
droplist_2 = ['subset_1']

In [ ]:
left_team = tracker.drop(droplist_1, axis = 1)
right_team = tracker.drop(droplist_2, axis = 1)

In [ ]:
left_team.shape

(15974, 10)

In [ ]:
right_team.shape

(15974, 10)

### Rename columns in each dframe 

In [ ]:
left_team.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_1
15970,2021,Geelong,77,Melbourne,81,Kardinia Park,GE,ME,-4,1
15971,2021,Carlton,75,GWS Giants,89,Docklands,CA,GW,-14,1
15972,2021,St Kilda,107,Fremantle,49,Bellerive Oval,SK,FR,58,1
15973,2021,Essendon,102,Collingwood,64,M.C.G.,ES,CO,38,1
15974,2021,Adelaide,98,North Melbourne,54,Adelaide Oval,AD,NM,44,1


In [ ]:
left_team.columns = ['year','team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']

In [ ]:
left_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15970,2021,Geelong,77,Melbourne,81,Kardinia Park,GE,ME,-4,1
15971,2021,Carlton,75,GWS Giants,89,Docklands,CA,GW,-14,1
15972,2021,St Kilda,107,Fremantle,49,Bellerive Oval,SK,FR,58,1
15973,2021,Essendon,102,Collingwood,64,M.C.G.,ES,CO,38,1
15974,2021,Adelaide,98,North Melbourne,54,Adelaide Oval,AD,NM,44,1


In [ ]:
right_team.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_2
15970,2021,Geelong,77,Melbourne,81,Kardinia Park,GE,ME,-4,0
15971,2021,Carlton,75,GWS Giants,89,Docklands,CA,GW,-14,0
15972,2021,St Kilda,107,Fremantle,49,Bellerive Oval,SK,FR,58,0
15973,2021,Essendon,102,Collingwood,64,M.C.G.,ES,CO,38,0
15974,2021,Adelaide,98,North Melbourne,54,Adelaide Oval,AD,NM,44,0


In [ ]:
right_team.columns = ['year','opp_team','op_pts','team','tm_pts','venue','op','tm','hvar','lr']

In [ ]:
right_team.tail()

,year,opp_team,op_pts,team,tm_pts,venue,op,tm,hvar,lr
15970,2021,Geelong,77,Melbourne,81,Kardinia Park,GE,ME,-4,0
15971,2021,Carlton,75,GWS Giants,89,Docklands,CA,GW,-14,0
15972,2021,St Kilda,107,Fremantle,49,Bellerive Oval,SK,FR,58,0
15973,2021,Essendon,102,Collingwood,64,M.C.G.,ES,CO,38,0
15974,2021,Adelaide,98,North Melbourne,54,Adelaide Oval,AD,NM,44,0


### Now re-order the second dframe to match columns

### Note double brackets to re-order

In [ ]:
right_team = right_team[['year', 'team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']]
right_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15970,2021,Melbourne,81,Geelong,77,Kardinia Park,ME,GE,-4,0
15971,2021,GWS Giants,89,Carlton,75,Docklands,GW,CA,-14,0
15972,2021,Fremantle,49,St Kilda,107,Bellerive Oval,FR,SK,58,0
15973,2021,Collingwood,64,Essendon,102,M.C.G.,CO,ES,38,0
15974,2021,North Melbourne,54,Adelaide,98,Adelaide Oval,NM,AD,44,0


In [ ]:
left_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15970,2021,Geelong,77,Melbourne,81,Kardinia Park,GE,ME,-4,1
15971,2021,Carlton,75,GWS Giants,89,Docklands,CA,GW,-14,1
15972,2021,St Kilda,107,Fremantle,49,Bellerive Oval,SK,FR,58,1
15973,2021,Essendon,102,Collingwood,64,M.C.G.,ES,CO,38,1
15974,2021,Adelaide,98,North Melbourne,54,Adelaide Oval,AD,NM,44,1


## CONCATENATION

### Now very easily, can concatenate the two dframes

In [ ]:
merge_ft = pd.concat([left_team, right_team], axis=0)
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1


In [ ]:
merge_ft.shape

(31948, 10)

In [ ]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15970,2021,Melbourne,81,Geelong,77,Kardinia Park,ME,GE,-4,0
15971,2021,GWS Giants,89,Carlton,75,Docklands,GW,CA,-14,0
15972,2021,Fremantle,49,St Kilda,107,Bellerive Oval,FR,SK,58,0
15973,2021,Collingwood,64,Essendon,102,M.C.G.,CO,ES,38,0
15974,2021,North Melbourne,54,Adelaide,98,Adelaide Oval,NM,AD,44,0


In [ ]:
merge_ft.sort_index(inplace=True)
merge_ft.head(10)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
3,1897,Essendon,47,Geelong,24,Corio Oval,ES,GE,-23,0
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
4,1897,Melbourne,44,South Melbourne,27,Lake Oval,ME,SM,-17,0
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1
5,1897,Carlton,36,South Melbourne,40,Lake Oval,CA,SM,4,0


## Create team result column

In [ ]:
def neg_hvar(hvar):
        return -1*hvar

In [ ]:
merge_ft["neg_dummy"] = merge_ft['hvar'].apply(neg_hvar)

In [ ]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy
15972,2021,St Kilda,107,Fremantle,49,Bellerive Oval,SK,FR,58,1,-58
15973,2021,Collingwood,64,Essendon,102,M.C.G.,CO,ES,38,0,-38
15973,2021,Essendon,102,Collingwood,64,M.C.G.,ES,CO,38,1,-38
15974,2021,Adelaide,98,North Melbourne,54,Adelaide Oval,AD,NM,44,1,-44
15974,2021,North Melbourne,54,Adelaide,98,Adelaide Oval,NM,AD,44,0,-44


In [ ]:
def team_var(hvar):
    if merge_ft['lr'] == 1:
        return hvar
    else: return neg_dummy

In [ ]:
def new_lr(lr):
    if lr == 0:
        return -1
    else: return 1

In [ ]:
merge_ft["new_lr"] = merge_ft['lr'].apply(new_lr)
merge_ft["tm_var"] = merge_ft['new_lr']*merge_ft['hvar']

In [ ]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy,new_lr,tm_var
15972,2021,St Kilda,107,Fremantle,49,Bellerive Oval,SK,FR,58,1,-58,1,58
15973,2021,Collingwood,64,Essendon,102,M.C.G.,CO,ES,38,0,-38,-1,-38
15973,2021,Essendon,102,Collingwood,64,M.C.G.,ES,CO,38,1,-38,1,38
15974,2021,Adelaide,98,North Melbourne,54,Adelaide Oval,AD,NM,44,1,-44,1,44
15974,2021,North Melbourne,54,Adelaide,98,Adelaide Oval,NM,AD,44,0,-44,-1,-44


### Can now remove the columns new_lr and neg_dummy, which were used to create tm_var

In [ ]:
merge_ft = merge_ft.drop(['neg_dummy', 'new_lr'], axis=1)

In [ ]:
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23


## Export as csv

In [ ]:
merge_ft.to_csv('all_results_to_2021_R23.csv')